In [1]:
import os
import numpy as np

# Function to load dataset from file. This is needed so we can easily load the two datasets without copy pasteing.
def load_data( name ):
    X_train = np.load(os.path.join('data', name, name + '_train_vectors.npy'))
    X_test = np.load(os.path.join('data', name, name + '_test_vectors.npy'))
    Y_train = np.load(os.path.join('data', name, name + '_train_labels.npy'))
    Y_test = np.load(os.path.join('data', name, name + '_test_labels.npy'))

    # The images need to have shape (28, 28, 1), we didn't take care of this in preprocessing.
    X_train = np.expand_dims(X_train, -1)
    X_test = np.expand_dims(X_test, -1)

    return X_train, Y_train, X_test, Y_test

# The same model is used for both datasets so it is more convenient to make them in a funtion.
def make_model(X_train, Y_train, batch_selection, log_file):

    # This is a simple convolutional neural network. It isn't the best possible network for MNIST
    # but the point here is to test how much batch selection methods will speed up a CNN, not the CNN itself.
    model = Sequential()
    model.add(layers.Input(shape = (28, 28, 1,)))
    model.add(layers.Conv2D(64, kernel_size = (3, 3), activation = "relu"))
    model.add(layers.MaxPooling2D( pool_size = (2, 2)))
    model.add(layers.Conv2D(64, kernel_size = (3, 3), activation = "relu"))
    model.add(layers.MaxPooling2D(pool_size = (2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(10, activation = "softmax"))
    
    # Put the model in our custom training loop.
    training = TrainingLoop(
        model = model,
        X = X_train,
        y = Y_train,
        optimizer = keras.optimizers.Adam(),
        loss_function = keras.losses.CategoricalCrossentropy(from_logits=True),
        batch_size = 64,
        train_metrics = tf.keras.metrics.CategoricalAccuracy(),
        val_metrics = tf.keras.metrics.CategoricalAccuracy(),
        validation_split = 0.2,
        batch_selection = batch_selection,
        log_file = "logs/" + log_file + "/MNIST.csv",
    )

    # We still have to compile the model for the test evaluation.
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])

    return model, training


In [2]:
from loop import TrainingLoop
import tensorflow as tf

# These lines will make the gpu not give errors.
gpus= tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


In [3]:
from batchSelection import bad_batch_selector

selectorList = [[None, 'original'], [bad_batch_selector, 'windowed']]

X_train, Y_train, X_test, Y_test = load_data( "mnist" )
for selector in selectorList:
    print( "\n\n" + selector[1] + "\n")
    # Set random seed so the comparison of different solutions won't be affected by it.
    tf.random.set_seed(42)
    np.random.seed(42)
    
    model, training = make_model( X_train, Y_train, selector[0], selector[1] )

    training.train(epochs = 20)




original

Epoch 1/20	Loss: 1.5074	Metrics: 0.9157: 	Validation metrics: 0.9672: 	100% | 750/750 [00:03<00:00, 195.70it/s]
Epoch 2/20	Loss: 1.4951	Metrics: 0.9711: 	Validation metrics: 0.9746: 	100% | 750/750 [00:02<00:00, 285.24it/s]
Epoch 3/20	Loss: 1.4916	Metrics: 0.9792: 	Validation metrics: 0.9798: 	100% | 750/750 [00:02<00:00, 289.06it/s]
Epoch 4/20	Loss: 1.4835	Metrics: 0.9827: 	Validation metrics: 0.9801: 	100% | 750/750 [00:02<00:00, 288.75it/s]
Epoch 5/20	Loss: 1.4797	Metrics: 0.9852: 	Validation metrics: 0.9824: 	100% | 750/750 [00:02<00:00, 286.88it/s]
Epoch 6/20	Loss: 1.4767	Metrics: 0.9859: 	Validation metrics: 0.9831: 	100% | 750/750 [00:02<00:00, 284.06it/s]
Epoch 7/20	Loss: 1.4758	Metrics: 0.9879: 	Validation metrics: 0.9840: 	100% | 750/750 [00:02<00:00, 289.01it/s]
Epoch 8/20	Loss: 1.4762	Metrics: 0.9894: 	Validation metrics: 0.9862: 	100% | 750/750 [00:02<00:00, 282.86it/s]
Epoch 9/20	Loss: 1.4780	Metrics: 0.9904: 	Validation metrics: 0.9847: 	100% | 750/750 [00:02